# Task 2

In [98]:
import numpy as np
import torch
import cv2
import matplotlib.pyplot as plt
import webcolors

In [ ]:
model  = torch.hub.load('ultralytics/yolov5','yolov5l',pretrained=True)

In [62]:
img1 = cv2.imread('mr1.jpeg')

In [63]:
print(model(img1))

image 1/1: 275x183 9 persons, 6 chairs, 1 dining table
Speed: 7.7ms pre-process, 1042.7ms inference, 0.0ms NMS per image at shape (1, 3, 640, 448)


In [65]:
img2 = cv2.imread('mr5.jpeg')
model(img2)

In [67]:
img3 = cv2.imread('tf1.jpg')
model(img3)

In [196]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the model
age_sex_model = load_model('Age_Sex_Detection.keras')


In [198]:
def preprocess_image(image):
    # Convert the image to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Resize the image to the size expected by the model, e.g., 64x64
    image = cv2.resize(image, (48, 48))
    
    # Normalize the image
    image = image / 255.0
    
    # Expand dimensions to match the model input
    image = np.expand_dims(image, axis=0)
    
    return image


In [215]:
def predict_sex(image, model = age_sex_model):
    image = preprocess_image(image)
    outputs = model.predict(image)
    sex, age = outputs  # Assuming the output is [age, sex]
    sex = 'Male' if sex <= 0.5 else 'Female'  # Threshold for sex classification
    return sex

In [ ]:
def detect_dominant_color(image):
    # Convert the image to RGB color space
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Reshape the image to be a list of pixels
    pixels = image.reshape((-1, 3))
    
    # Convert to float
    pixels = np.float32(pixels)
    
    # Define criteria and apply kmeans()
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
    k = 1
    _, labels, palette = cv2.kmeans(pixels, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    
    # Find the most frequent color
    dominant_color = palette[np.argmax(np.bincount(labels.flatten()))]
    
    return dominant_color

In [ ]:
def get_color_category(rgb_color):
    r, g, b = rgb_color
    if r > 70 and g < 100 and b < 100 and r>g and r>b:
        return 'red'
    elif r < 100 and g < 100 and b > 70 and b>g and b>r:
        return 'blue'
    else:
        return 'other'

In [ ]:
def detect(img_path):
    img = cv2.imread(img_path)
    results = model(img)
    print(results)
    locs = results.pandas().xyxy[0]
    car_cnt = 0
    male_cnt = 0
    female_cnt = 0
    other_vehicle_no = 0
    for i in range(len(locs)):
        xmin,ymin,xmax,ymax,obj = int(locs['xmin'][i]),int(locs['ymin'][i]),int(locs['xmax'][i]),int(locs['ymax'][i]),locs['name'][i]
        if obj == 'car':
            car_cnt += 1
            color = detect_dominant_color(img[ymin:ymax,xmin:xmax])
            color = get_color_category(color)
            if color == 'blue':
                color = 'red'
            elif color == 'red':
                color = 'blue'
            label = f'{'Car'} {color}'
            cv2.putText(img, label, (int(xmin), int(ymax)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        if obj == 'person':
            person_img = img[ymin:ymax,xmin:xmax]
            sex = predict_sex(person_img)
            if sex == 'Male':
                male_cnt +=1
            else:
                female_cnt +=1
            label = f'{'Person'} {sex}'
            cv2.putText(img, label, (int(xmin), int(ymax)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        cv2.rectangle(img,(xmin,ymin),(xmax,ymax),(255, 0, 0),1,1)
    other_vehicle_no = len(locs) - car_cnt - male_cnt - female_cnt
    print('No of Cars: ',car_cnt)
    print('No of Male: ',male_cnt)
    print('No of Female: ',female_cnt)
    print('Other Vehicles: ',other_vehicle_no)
    cv2.imwrite(img_path.split('.')[0]+'new'+'.jpeg',img)

In [224]:
detect('tf1.jpg')

image 1/1: 427x640 10 persons, 23 cars, 3 motorcycles, 2 buss, 4 trucks
Speed: 13.2ms pre-process, 1385.9ms inference, 4.1ms NMS per image at shape (1, 3, 448, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
No of Cars:  23
No of Male:  9
No of Female:  1
Other Vehicles:  9


In [139]:
img_path = 'tf2.jpeg'
detect(img_path)

image 1/1: 164x308 8 cars
Speed: 12.9ms pre-process, 1144.1ms inference, 33.7ms NMS per image at shape (1, 3, 352, 640)


In [189]:
detect('tf3.jpeg')

image 1/1: 183x275 7 cars, 1 truck
Speed: 9.5ms pre-process, 1012.5ms inference, 0.0ms NMS per image at shape (1, 3, 448, 640)


In [190]:
detect('tf4.jpeg')

image 1/1: 168x300 8 cars, 2 buss
Speed: 13.3ms pre-process, 885.3ms inference, 0.0ms NMS per image at shape (1, 3, 384, 640)


In [218]:
detect('tf5.jpg')

image 1/1: 1063x1600 29 persons, 15 cars, 22 motorcycles
Speed: 8.0ms pre-process, 1320.5ms inference, 5.1ms NMS per image at shape (1, 3, 448, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━

In [176]:
detect('redcar.jpeg')

image 1/1: 126x400 1 car
Speed: 2.4ms pre-process, 673.8ms inference, 2.0ms NMS per image at shape (1, 3, 224, 640)
red


In [179]:
detect('bluecar.jpg')

image 1/1: 956x1300 1 car
Speed: 14.0ms pre-process, 1063.4ms inference, 11.5ms NMS per image at shape (1, 3, 480, 640)
